In [ ]:
# Install pyspark
!pip install pyspark

from pyspark import *
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import *
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

## Basic example on Transformer and Estimator

In [ ]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# Prepare training data from a list of (label, features) tuples.
# Dense Vectors are just NumPy arrays

training = spark.createDataFrame([
    (1, Vectors.dense([0.0, 1.1, 0.1])),
    (0, Vectors.dense([2.0, 1.0, -1.0])),
    (0, Vectors.dense([2.0, 1.3, 1.0])),
    (1, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])
# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)

# Print out the parameters, documentation, and any default values.
print(lr.explainParams())

In [ ]:
# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

print(model1)
# model1 is a Model (i.e., a transformer produced by an Estimator)
print("Model 1's trained coefficients: ", model1.coefficients)

In [ ]:
# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap[lr.probabilityCol] = "myProbability"  # Change output column name

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMap)
print("Model 2's trained coefficients: ", model2.coefficients)

In [ ]:
# Prepare test data
test = spark.createDataFrame([
    (1, Vectors.dense([-1.0, 1.5, 1.3])),
    (2, Vectors.dense([3.0, 2.0, -0.1])),
    (3, Vectors.dense([0.0, 2.2, -1.5]))], ["id", "features"])

# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.

model1.transform(test).show()
model2.transform(test).show()

## Pipeline example

In [ ]:
# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d spark spark", 1),
    (1, "b d", 0),
    (2, "spark f g h", 1),
    (3, "hadoop mapreduce", 0)
], ["id", "text", "label"])

In [ ]:
# A tokenizer converts the input string to lowercase and then splits it by white spaces.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
tokenizer.transform(training).show()

In [ ]:
# The same can be achieved by DataFrameAPI:
# But you will need to wrap it as a transformer to use it in a pipeline.

training.select('*', split(training['text'],' ').alias('words')).show()

In [ ]:
# Maps a sequence of terms to their term frequencies using the hashing trick.
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
a = hashingTF.transform(tokenizer.transform(training))
a.show(truncate=False)

print(a.select('features').first())

In [ ]:
# lr is an estimator
lr = LogisticRegression(maxIter=10, regParam=0.001)

# Now we are ready to assumble the pipeline
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
                            
# Fit the pipeline to training documents.
model = pipeline.fit(training)

# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents and print columns of interest.
model.transform(test).show()

## Example: Summary statistics

In [ ]:
import numpy as np

from pyspark.mllib.stat import Statistics

mat = sc.parallelize(
    [np.array([1.0, 10.0, 100.0]), np.array([2.0, 20.0, 200.0]), np.array([3.0, 30.0, 300.0])]
)  # an RDD of Vectors

# Compute column summary statistics.
summary = Statistics.colStats(mat)
print(summary.mean())  # a dense vector containing the mean value for each column
print(summary.variance())  # column-wise variance
print(summary.numNonzeros())  # number of nonzeros in each column

## Example: Correlations

In [ ]:
from pyspark.mllib.stat import Statistics

seriesX = sc.parallelize([1.0, 2.0, 3.0, 3.0, 5.0])  # a series
# seriesY must have the same number of partitions and cardinality as seriesX
seriesY = sc.parallelize([11.0, 22.0, 33.0, 33.0, 555.0])

# Compute the correlation using Pearson's method. Enter "spearman" for Spearman's method.
# If a method is not specified, Pearson's method will be used by default.
print("Correlation is: " + str(Statistics.corr(seriesX, seriesY, method="pearson")))

data = sc.parallelize(
    [np.array([1.0, 10.0, 100.0]), np.array([2.0, 20.0, 200.0]), np.array([5.0, 33.0, 366.0])]
)  # an RDD of Vectors

# calculate the correlation matrix using Pearson's method. Use "spearman" for Spearman's method.
# If a method is not specified, Pearson's method will be used by default.
print(Statistics.corr(data, method="pearson"))


## Example: Classification by Decision Tree

The example below demonstrates how to load a LIBSVM data file, parse it as an RDD of LabeledPoint and then perform classification using a decision tree with Gini impurity as an impurity measure and a maximum tree depth of 5. The test error is calculated to measure the algorithm accuracy.

In [ ]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, 'sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [ ]:
# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)


In [ ]:
# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())